In [1]:
 !pip install rioxarray "xarray[complete]" geopandas  "dask[complete]"

  Obtaining dependency information for rioxarray from https://files.pythonhosted.org/packages/4e/34/8ee402df8a49228b6aae89ab2127cdd5aa0ba0976904af7d55270f4f30b8/rioxarray-0.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for xarray[complete] from https://files.pythonhosted.org/packages/cc/21/c3a9d3017c131cea3c2967957b05448ffc81cafced239796bbbdc98073e4/xarray-2023.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/cb/7b/2fb6ab376c78d2e60812ad0bf4b24889309d2aeb543f54efcd586c69907f/geopandas-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for rasterio>=1.2 from https://files.pythonhosted.org/packages/c8/7e/08d80abba94a41a6102b432897fe21e99896120fad2766b4875bfc06a8d2/rasterio-1.3.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pyproj>=2.2 from https://files.pythonhosted.org/packages/24/18/0a0895fd8c1136fea0aed5e0ee290e59

In [1]:
import rioxarray as rx
import pandas as pd
import boto3
import xarray as xr
import glob
import shutil
import os
import zarr
import s3fs
import geopandas as gpd
from dask.diagnostics import ProgressBar

In [2]:
pathS3 = "s3://climate-action-datalake/zone=raw/source=chirps"
localPath = 'data/toZarr/*.nc'

In [6]:
chirps = xr.open_dataset('data/toZarr/19810111.nc')

In [8]:
chirps.to_zarr('sample.zarr')

In [14]:
chirps_v2 = xr.open_dataset('sample.zarr')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'h5netcdf' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


In [15]:
chirps_v2

<xarray.Dataset>
Dimensions:        (lat: 2000, lon: 7200, time: 1, band: 1)
Coordinates:
  * lat            (lat) float64 49.97 49.92 49.87 ... -49.88 -49.93 -49.98
  * lon            (lon) float64 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time           (time) datetime64[ns] 1981-01-11
Dimensions without coordinates: band
Data variables:
    precipitation  (time, band, lat, lon) float32 ...

In [3]:
def convertKelvinCelsius(k):
    c = k - 273.15
    return c

In [3]:
allFiles = xr.open_mfdataset(localPath,parallel=True)

In [4]:
print('Hola mundo')

Hola mundo


In [6]:
#allFiles = allFiles.apply(convertKelvinCelsius)

In [ ]:
allFiles

In [23]:
compressor = zarr.Blosc(cname='zstd')
encoding = {vname: {'compressor': compressor,'chunks': (2000,7200,1,1)} for vname in allFiles.data_vars}


In [24]:
s3 = s3fs.S3FileSystem()
store = s3fs.S3Map(root="s3://climate-action-datalake/zone=raw/source=chirps/variable=precipitation.za", s3=s3, check=False)

In [25]:
with ProgressBar():
    result = allFiles.to_zarr(store=store, encoding=encoding, consolidated=True)

NotImplementedError: Specified zarr chunks encoding['chunks']=(2000, 7200, 1, 1) for variable named 'precipitation' would overlap multiple dask chunks ((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
print("He finalizado")